# Model Fit in Linear Regression - Lab

## Introduction
In this lab, you'll learn how to evaluate your model results, and you'll learn methods to select the appropriate features using stepwise selection.

## Objectives
You will be able to:
* Analyze the results of regression and R-squared and adjusted-R-squared 
* Understand and apply forward and backward predictor selection

## The Boston Housing Data once more

We pre-processed the Boston Housing Data the same way we did before:

- We dropped "ZN" and "NOX" completely
- We categorized "RAD" in 3 bins and "TAX" in 4 bins
- We transformed "RAD" and "TAX" to dummy variables and dropped the first variable
- We used min-max-scaling on "B", "CRIM" and "DIS" (and logtransformed all of them first, except "B")
- We used standardization on "AGE", "INDUS", "LSTAT" and "PTRATIO" (and logtransformed all of them first, except for "AGE") 

In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
boston_features = boston_features.drop(["NOX","ZN"],axis=1)

# first, create bins for based on the values observed. 3 values will result in 2 bins
bins = [0,6,  24]
bins_rad = pd.cut(boston_features['RAD'], bins)
bins_rad = bins_rad.cat.as_unordered()

# first, create bins for based on the values observed. 4 values will result in 3 bins
bins = [0, 270, 360, 712]
bins_tax = pd.cut(boston_features['TAX'], bins)
bins_tax = bins_tax.cat.as_unordered()

tax_dummy = pd.get_dummies(bins_tax, prefix="TAX", drop_first=True)
rad_dummy = pd.get_dummies(bins_rad, prefix="RAD", drop_first=True)
boston_features = boston_features.drop(["RAD","TAX"], axis=1)
boston_features = pd.concat([boston_features, rad_dummy, tax_dummy], axis=1)

age = boston_features["AGE"]
b = boston_features["B"]
logcrim = np.log(boston_features["CRIM"])
logdis = np.log(boston_features["DIS"])
logindus = np.log(boston_features["INDUS"])
loglstat = np.log(boston_features["LSTAT"])
logptratio = np.log(boston_features["PTRATIO"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["CRIM"] = (logcrim-min(logcrim))/(max(logcrim)-min(logcrim))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["AGE"] = (age-np.mean(age))/np.sqrt(np.var(age))
boston_features["INDUS"] = (logindus-np.mean(logindus))/np.sqrt(np.var(logindus))
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))
boston_features["PTRATIO"] = (logptratio-np.mean(logptratio))/(np.sqrt(np.var(logptratio)))

# y-values
boston_target = pd.DataFrame(boston.target)
new_columns = ['medv']
boston_target.columns = new_columns

#dataframe that includes y-values
boston_df = pd.concat([boston_features, boston_target], axis=1)
new_columns = ['CRIM', 'INDUS', 'CHAS', 'RM', 'AGE', 'DIS', 'PTRATIO', 'B', 'LSTAT', 'RAD_6to24', 'TAX_270to360', 'TAX_360to712', 'medv']
boston_df.columns = new_columns
# boston_df.head()

## Perform stepwise selection

The code for stepwise selection is copied below. Use this code provided on your preprocessed Boston Housing Data.

In [4]:
import statsmodels.api as sm

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [5]:
# Your code here
stepwise_selection(boston_df.drop('medv', axis=1), boston_df['medv'], 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True)

Add  LSTAT                          with p-value 9.27989e-122
Add  RM                             with p-value 1.98621e-16
Add  PTRATIO                        with p-value 2.5977e-12
Add  DIS                            with p-value 2.85496e-09
Add  B                              with p-value 2.77572e-06
Add  INDUS                          with p-value 0.0017767
Add  CHAS                           with p-value 0.0004737


['LSTAT', 'RM', 'PTRATIO', 'DIS', 'B', 'INDUS', 'CHAS']

### Build the final model again in Statsmodels

In [6]:
# Your code here
import statsmodels.api as sm
from statsmodels.formula.api import ols

# formula = "mdev ~ CRIM+INDUS+CHAS+RM+AGE+DIS+PTRATIO+B+LSTAT+RAD_(6, 24]+TAX_(270, 360]+TAX_(360, 712]"
outcome = 'medv'
predictors = boston_df.drop('medv', axis=1)
pred_sum = "+".join(predictors.columns)
formula = outcome + "~" + pred_sum
model = ols(formula= formula, data=boston_df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   medv   R-squared:                       0.779
Model:                            OLS   Adj. R-squared:                  0.774
Method:                 Least Squares   F-statistic:                     144.9
Date:                Tue, 17 Sep 2019   Prob (F-statistic):          5.08e-153
Time:                        19:34:45   Log-Likelihood:                -1458.2
No. Observations:                 506   AIC:                             2942.
Df Residuals:                     493   BIC:                             2997.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        8.6442      3.189      2.711      0.007       2.379      14.910
CRIM            -1.9538      2.115     -0.924      0.356      -6.110       2.202
INDUS           -0.8046      0.362     -2.220      0.027      -1.517      -0.093
CHAS             2.5959      0.796      3.260      0.001       1.032       4.160
RM               2.6466      0.408      6.488      0.000       1.845       3.448
AGE              0.0794      0.352      0.226      0.821      -0.612       0.770
DIS            -10.0962      1.856     -5.439      0.000     -13.743      -6.449
PTRATIO         -1.4867      0.241     -6.160      0.000      -1.961      -1.013
B                3.8412      0.986      3.897      0.000       1.905       5.778
LSTAT           -5.6288      0.354    -15.912      0.000      -6.324      -4.934
RAD_6to24        1.3380      0.672      1.990      0.047       0.017       2.659
TAX_270to360    -1.2598      0.600     -2.100      0.036      -2.438      -0.081
TAX_360to712    -2.1461      0.704     -3.047      0.002      -3.530      -0.762
==============================================================================
Omnibus:                      106.730   Durbin-Watson:                   1.093
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              432.101
Skew:                           0.891   Prob(JB):                     1.48e-94
Kurtosis:                       7.162   Cond. No.                         117.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The stepwise procedure mentions that "INDUS" was added with a p-value of 0.0017767, but our statsmodels output returns a p-value of 0.000. Use some of the stepwise procedure logic to find the intuition behind this!

## Use Feature ranking with recursive feature elimination

Use feature ranking to select the 5 most important features

In [11]:
# Your code here
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()
selector = RFE(linreg, n_features_to_select = 5)
selector = selector.fit(predictors, boston_df['medv'])
print(selector.support_) #top_5 = ['CHAS', 'RM', 'DIS', 'B', 'LSTAT']
# print(boston_df.columns)

[False False  True  True False  True False  True  True False False False]


Fit the linear regression model again using the 5 columns selected

In [18]:
# Your code here
top1 = boston_df['CHAS']
top2 = boston_df['RM']
top3 = boston_df['DIS']
top4 = boston_df['B']
top5 = boston_df['LSTAT']
top_5 = pd.concat([top1, top2, top3, top4, top5, boston_target], axis=1)
# top_5
outcome = 'medv'
predictors = top_5.drop(columns='medv', axis=1)
pred_sum = "+".join(predictors.columns)
# pred_sum
formula = outcome + "~" + pred_sum
# formula
model = ols(formula= formula, data=top_5).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   medv   R-squared:                       0.743
Model:                            OLS   Adj. R-squared:                  0.740
Method:                 Least Squares   F-statistic:                     289.1
Date:                Tue, 17 Sep 2019   Prob (F-statistic):          5.96e-145
Time:                        19:46:06   Log-Likelihood:                -1496.5
No. Observations:                 506   AIC:                             3005.
Df Residuals:                     500   BIC:                             3030.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.4974      2.867     -0.174      0.862      -6.130       5.135
CHAS           2.9350      0.834      3.518      0.000       1.296       4.574
RM             3.4372      0.405      8.497      0.000       2.642       4.232
DIS           -6.5804      1.116     -5.894      0.000      -8.774      -4.387
B              4.6536      0.989      4.707      0.000       2.711       6.596
LSTAT         -6.2522      0.330    -18.951      0.000      -6.900      -5.604
==============================================================================
Omnibus:                       82.869   Durbin-Watson:                   1.032
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              269.427
Skew:                           0.743   Prob(JB):                     3.12e-59
Kurtosis:                       6.251   Cond. No.                         91.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Now, predict $\hat y$ using your model. you can use `.predict()` in scikit-learn

In [20]:
# Your code here
predicted_y = model.predict()

Now, using the formulas of R-squared and adjusted-R-squared below, and your Python/numpy knowledge, compute them and contrast them with the R-squared and adjusted-R-squared in your statsmodels output using stepwise selection. Which of the two models would you prefer?

$SS_{residual} = \sum (y - \hat{y})^2 $

$SS_{total} = \sum (y - \bar{y})^2 $

$R^2 = 1- \dfrac{SS_{residual}}{SS_{total}}$

$R^2_{adj}= 1-(1-R^2)\dfrac{n-1}{n-p-1}$

In [24]:
# Your code here

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
style.use('Solarize_Light2')
%matplotlib inline

def r_squared(y_real, y_predicted):
    import numpy as np
    ssres = ((y_real - y_predicted)**2).sum()
    sstot = ((y_real - (y_real.mean()))**2).sum()
    r_squ = 1 -(ssres / sstot)
    return r_squ

print('r-squared: ' + str(r_squared(top_5['medv'], predicted_y)))
# r_squared is 0.742981  

def r_squared_adj(y_real, y_predicted, num_coeff):
    import numpy as np
    ssres = ((y_real - y_predicted)**2).sum()
    sstot = ((y_real - (y_real.mean()))**2).sum()
    r_squ = 1 -(ssres / sstot)
    r_squ_adj = (1 - ((1-r_squ)*((len(y_real)-1) / (len(y_real)-num_coeff-1))))
    return r_squ_adj
print('r-squared adjusted: ' + str(r_squared_adj(top_5['medv'], predicted_y, 5)))
# adjusted_r_squared is 0.740411

r-squared: 0.7429807743129864
r-squared adjusted: 0.7404105820561162


## Level up - Optional

- Perform variable selection using forward selection, using this resource: https://planspace.org/20150423-forward_selection_with_statsmodels/. Note that this time features are added based on the adjusted-R-squared!
- Tweak the code in the `stepwise_selection()`-function written above to just perform forward selection based on the p-value.

## Summary
Great! You now performed your own feature selection methods!